#### FETCH OLIST (List of Faculties)  
#### EXPORT AS CSV

In [ ]:
import requests
import pandas as pd

def fetch_olist_dataframe(
    language: str,
    csv_path: str = None
) -> pd.DataFrame:
    """
    Fetches the OListSet from UZH Student Services in the specified language,
    returns a DataFrame with columns 'OObjId' and 'OText',
    and optionally writes it out to a CSV file.

    Parameters
    ----------
    language : str
        Two‐letter language code, must be either 'EN' or 'DE'.
    csv_path : str, optional
        If provided (e.g. "olist_de.csv"), the DataFrame will be saved to this path.
        If None (default), no CSV is written.

    Returns
    -------
    pd.DataFrame
        A DataFrame containing the OObjId and OText for each entry.

    Raises
    ------
    ValueError
        If `language` is not 'EN' or 'DE'.
    requests.HTTPError
        If the HTTP request fails (e.g. non‐200 status code).
    KeyError
        If the expected keys 'OObjId' and 'OText' are not found in the JSON.
    """
    # 1) Validate language
    language = language.upper()
    if language not in ("EN", "DE"):
        raise ValueError("language must be either 'EN' or 'DE'")

    # 2) Build URL with correct language parameter
    url = (
        "https://studentservices.uzh.ch/sap/opu/odata/uzh/vvz_data_srv/"
        f"OListSet?sap-client=001&sap-language={language}&$format=json"
    )

    # 3) Fetch JSON from the endpoint
    response = requests.get(url)
    response.raise_for_status()   # Raise an exception if request failed

    # 4) Parse JSON
    payload = response.json()
    results = payload.get("d", {}).get("results", [])

    # 5) Build DataFrame and check for expected columns
    df = pd.DataFrame(results)
    if "OObjId" not in df.columns or "OText" not in df.columns:
        raise KeyError("Expected keys 'OObjId' and 'OText' not found in JSON response")

    df = df[["OObjId", "OText"]]

    # 6) If csv_path is given, export to CSV
    if csv_path:
        # Use index=False so that pandas does not write the DataFrame index as a column
        df.to_csv(csv_path, index=False)
        print(f"DataFrame successfully written to '{csv_path}'")

    return df


# ----------------------------
# Example usage:
if __name__ == "__main__":
    # 1) Fetch German DataFrame and export to CSV
    df_de = fetch_olist_dataframe("DE", csv_path="./csvs/olist_de.csv")
    print("First 5 rows (DE):")
    print(df_de.head(), "\n")

    # 2) Fetch English DataFrame without exporting
    df_en = fetch_olist_dataframe("EN", csv_path="./csvs/olist_en.csv")
    print("First 5 rows (EN):")
    print(df_en.head())


#### FETCH SCLIST (The degrees e.g Bachelor in ..., Masters in ...) for a given Organisation (Faculty)
#### EXPORT AS CSV

In [ ]:
import requests
import pandas as pd

def fetch_sc_list_dataframe(
    OObjId: str,
    PiqYear: str,
    PiqSession: str,
    language: str,
    export_csv: bool = False
) -> pd.DataFrame:
    """
    Fetches the OListSet entry (expanded with ToScList) for a given OObjId, PiqYear, PiqSession,
    and language ("EN" or "DE"). Returns a DataFrame with columns:
      - OObjId
      - PiqYear
      - PiqSession
      - ObjectId         (from each item under ToScList/results)
      - ExtDescription   (from each item under ToScList/results)

    Optionally, writes the DataFrame to a CSV named "ScList_<OObjId>.csv" if export_csv=True.

    Parameters
    ----------
    OObjId : str
        The Objekt‐ID (e.g. "50000003") to plug into the OListSet(...) key.
    PiqYear : str
        The PIQ year (e.g. "2024").
    PiqSession : str
        The PIQ session (e.g. "004").
    language : str
        Two‐letter language code, either "EN" or "DE".
    export_csv : bool, optional (default=False)
        If True, after building the DataFrame, write it out as "ScList_<OObjId>.csv"
        in the current working directory. If False, no file is created.

    Returns
    -------
    pd.DataFrame
        A DataFrame where each row corresponds to one entry under ToScList/results.
        If there are no sub‐entries, returns an empty DataFrame with the correct columns.

    Raises
    ------
    ValueError
        If `language` is not "EN" or "DE".
    requests.HTTPError
        If the HTTP request returns a non‐200 status code.
    KeyError
        If the JSON response does not contain the expected keys.
    """
    # 1) Validate language
    language = language.upper()
    if language not in ("EN", "DE"):
        raise ValueError("language must be either 'EN' or 'DE'")

    # 2) Build the URL (with $expand=ToScList)
    endpoint = (
        "https://studentservices.uzh.ch/sap/opu/odata/uzh/vvz_data_srv/"
        f"OListSet(OObjId='{OObjId}',PiqYear='{PiqYear}',PiqSession='{PiqSession}')"
        f"?sap-client=001&$expand=ToScList&sap-language={language}&$format=json"
    )

    # 3) Perform the HTTP GET
    response = requests.get(endpoint)
    response.raise_for_status()  # raises HTTPError if status != 200

    # 4) Parse JSON
    data = response.json()
    d_node = data.get("d", {})
    if not d_node:
        raise KeyError("The JSON payload did not contain a 'd' key.")

    to_sc_list = d_node.get("ToScList", {}).get("results", [])

    # 5) Build a list of rows
    rows = []
    for entry in to_sc_list:
        obj_id = entry.get("ObjectId", "")
        ext_desc = entry.get("ExtDescription", "")
        rows.append({
            "OObjId":        OObjId,
            "PiqYear":       PiqYear,
            "PiqSession":    PiqSession,
            "ObjectId":      obj_id,
            "ExtDescription": ext_desc
        })

    # 6) If there were no sub‐entries, return an empty DF with the correct columns
    columns = ["OObjId", "PiqYear", "PiqSession", "ObjectId", "ExtDescription"]
    if not rows:
        empty_df = pd.DataFrame(columns=columns)
        if export_csv:
            filename = f"ScList_{OObjId}.csv"
            empty_df.to_csv(filename, index=False)
            print(f"Empty DataFrame written to '{filename}' (no rows under ToScList).")
        return empty_df

    # 7) Otherwise, turn the list of dicts into a DataFrame
    df = pd.DataFrame(rows)

    # 8) Export to CSV if requested
    if export_csv:
        filename = f"./csvs/ScList_{language}_{OObjId}.csv"
        df.to_csv(filename, index=False)
        print(f"DataFrame successfully written to '{filename}'")

    return df


# --------------------------------------
# Example usage:
if __name__ == "__main__":
    # 1) Fetch German SC List for OObjId="50000003", PiqYear="2024", PiqSession="004", 
    #    and export it as "ScList_50000003.csv"
    df_sc_de = fetch_sc_list_dataframe(
        OObjId="50000003",
        PiqYear="2024",
        PiqSession="004",
        language="DE",
        export_csv=True
    )
    print("First 5 rows (DE):")
    print(df_sc_de.head())
    print(f"Total rows: {len(df_sc_de)}\n")

    # 2) Fetch English SC List without exporting
    df_sc_en = fetch_sc_list_dataframe(
        OObjId="50000003",
        PiqYear="2024",
        PiqSession="004",
        language="EN",
        export_csv=True
    )
    print("First 5 rows (EN):")
    print(df_sc_en.head())
    print(f"Total rows: {len(df_sc_en)}")


#### FETCH AND REMOVE UNECESSARY COLUMNS OF A SCDETAILS (Major in ... 150ects, Minor in ... 30 ects)
#### This is the list of cgs (majors, minors) that are linked to a certain Sc object(Bachelor of... or Masters in ...)
#### EXPORT AS CSV

In [ ]:
import requests
import pandas as pd

def fetch_sc_details_raw(
    ObjectId: str,
    PiqYear: str,
    PiqSession: str,
    language: str
) -> dict:
    """
    Fetches the raw JSON payload for ScDetailsSet(ObjectId, PiqYear, PiqSession)
    (with $expand=ToCgList) and returns the Python dictionary corresponding to the "d" node.

    Parameters
    ----------
    ObjectId : str
        The study‐program ID (e.g. "51085509").
    PiqYear : str
        The PIQ year (e.g. "2024").
    PiqSession : str
        The PIQ session (e.g. "004").
    language : str
        Two‐letter language code, either "EN" or "DE".

    Returns
    -------
    dict
        The raw JSON dictionary under the "d" key, containing all fields and
        the full ToCgList sub‐collection.

    Raises
    ------
    ValueError
        If `language` is not "EN" or "DE".
    requests.HTTPError
        If the HTTP request returns a non‐200 status code.
    KeyError
        If the JSON response is missing the "d" key.
    """
    language = language.upper()
    if language not in ("EN", "DE"):
        raise ValueError("language must be 'EN' or 'DE'")

    endpoint = (
        "https://studentservices.uzh.ch/sap/opu/odata/uzh/vvz_data_srv/"
        f"ScDetailsSet(ObjectId='{ObjectId}',PiqYear='{PiqYear}',PiqSession='{PiqSession}')"
        f"?sap-client=001&$expand=ToCgList&sap-language={language}&$format=json"
    )

    response = requests.get(endpoint)
    response.raise_for_status()

    payload = response.json()
    if "d" not in payload:
        raise KeyError("Expected key 'd' not found in JSON response")

    return payload["d"]


def clean_sc_details(
    raw_d: dict,
    language: str,
    export_csv: bool = False
) -> pd.DataFrame:
    """
    Takes the raw 'd' dictionary returned by fetch_sc_details_raw(...) and
    “cleans” it into a DataFrame with columns:
      - OObjectId   (parent faculty/faculty ID)
      - ObjectId    (same ID you passed in)
      - CgObjId     (each entry under ToCgList/results)
      - Text        (each entry’s “Text” field)
      - CatText     (each entry’s “CatText” field)

    If export_csv=True, writes a file named "ScDetails_<language>_<ObjectId>.csv"
    in the current working directory.

    Parameters
    ----------
    raw_d : dict
        The raw dictionary under "d" from fetch_sc_details_raw(...).
    language : str
        Two‐letter language code used to name the CSV (if export_csv=True).
    export_csv : bool, optional (default=False)
        If True, write the cleaned DataFrame to CSV.

    Returns
    -------
    pd.DataFrame
        A DataFrame where each row corresponds to one item under ToCgList/results,
        with columns ["OObjectId", "ObjectId", "CgObjId", "Text", "CatText"].
        If no ToCgList entries exist, returns an empty DataFrame with those columns.

    Raises
    ------
    KeyError
        If expected keys ("OObjId", etc.) are missing from raw_d.
    ValueError
        If `language` is not "EN" or "DE".
    """
    language = language.upper()
    if language not in ("EN", "DE"):
        raise ValueError("language must be 'EN' or 'DE'")

    # 1) Extract top‐level OObjectId and ensure it exists
    parent_OObjId = raw_d.get("OObjId", "")
    if not parent_OObjId:
        raise KeyError("Expected key 'OObjId' not found in raw data")

    # 2) Extract ObjectId, PiqYear, PiqSession from raw_d
    #    (these should match what was passed in)
    ObjectId_val    = raw_d.get("ObjectId", "")
    PiqYear_val     = raw_d.get("PiqYear", "")
    PiqSession_val  = raw_d.get("PiqSession", "")

    if not ObjectId_val:
        raise KeyError("Expected key 'ObjectId' not found in raw data")

    # 3) Get the list under ToCgList → results
    to_cg_list = raw_d.get("ToCgList", {}).get("results", [])

    # 4) Build a list of rows
    rows = []
    for entry in to_cg_list:
        cg_obj_id = entry.get("CgObjId", "")
        text       = entry.get("Text", "")
        cat_text   = entry.get("CatText", "")
        rows.append({
            "OObjectId": parent_OObjId,
            "ObjectId":  ObjectId_val,
            "CgObjId":   cg_obj_id,
            "Text":      text,
            "CatText":   cat_text
        })

    # 5) If there were no ToCgList entries, return an empty DF with the correct columns
    columns = ["OObjectId", "ObjectId", "CgObjId", "Text", "CatText"]
    if not rows:
        empty_df = pd.DataFrame(columns=columns)
        if export_csv:
            filename = f"./csvs/ScDetails_{language}_{ObjectId_val}.csv"
            empty_df.to_csv(filename, index=False)
            print(f"(Empty) DataFrame written to '{filename}' (no ToCgList entries).")
        return empty_df

    # 6) Otherwise, convert rows into a DataFrame
    df = pd.DataFrame(rows, columns=columns)

    # 7) Export to CSV if requested
    if export_csv:
        filename = f"./csvs/ScDetails_{language}_{ObjectId_val}.csv"
        df.to_csv(filename, index=False)
        print(f"Cleaned DataFrame written to '{filename}'")

    return df


if __name__ == "__main__":
    """
    Main routine example:
      1. Specify ObjectId, PiqYear, PiqSession, language, and export_csv flag.
      2. Call fetch_sc_details_raw(...) to get unmodified JSON.
      3. Call clean_sc_details(...) to flatten into a DataFrame.
      4. Print or save the DataFrame.
    """

    # 1) Set parameters (can easily be replaced by input() or argparse)
    ObjectId   = "51085509"
    PiqYear    = "2024"
    PiqSession = "004"
    language   = "EN"
    export_csv = True  # Change to False if you don't want CSV output

    # 2) Fetch raw "d" dictionary
    try:
        raw_data = fetch_sc_details_raw(
            ObjectId    = ObjectId,
            PiqYear     = PiqYear,
            PiqSession  = PiqSession,
            language    = language
        )
    except Exception as e:
        print("Error fetching raw SC details:", e)
        exit(1)

    # 3) Clean the raw data into a DataFrame
    try:
        df_clean = clean_sc_details(
            raw_d      = raw_data,
            language   = language,
            export_csv = export_csv
        )
    except Exception as e:
        print("Error cleaning SC details:", e)
        exit(1)

    # 4) Display the resulting DataFrame
    print("\n=== Final SC Details DataFrame ===")
    print(df_clean.to_string(index=False))


#### FETCH AND REMOVE UNECESSARY COLUMNS OF A CG DETAILS (Major in ... Minor in...)
#### EXPORT AS CSV
##### DESCRIPTION OF THE FIELDS:
MainLanguageDescription: main language  

FurtherLanguageDescription: other languages  

AdmissionRqDescription: Text that says the study service decide if you are accepted and link to the website to admission for a bachelor: https://www.uzh.ch/en/studies/application/bachelor.html  

Text: Name of the major/minor  

PointsText: Number of ECTS (150.0000)  

CatText: Category + ECTS (Major 150)  

CommonDescription : Composition of the Bachelor Degree, Quickly says fields covered  

DegreeDescription: Says that (for bachelor) give basic knowledge and allow to then pursue a career or start a master  

GradesDescription: Explains the scale from 1 to 6 with pass at 4, half and quarter points exist  

VistasDescription: Explain the fields you will be good at and can work in, says (for bachelor) that you can also do a masters after  

StructureDescription: Details on structure so assesment level 60ECTS, advanced 90ects, bachelr thesis is 18 (CHECK FOR OVERLAP OF INFOS WITH CommonDescription), says which title you are awarded when you finish (Bachelor of Arts UZH in Business and Economics) and useless link I think  

RulesDescription: contains the link https://www.oec.uzh.ch/en/studies/general/regulations.html  

AdviceDescription: contains the link http://www.oec.uzh.ch/en/advising or name and email of a person

IsTextEditable: True/False, don't know what it is  

HasChildConcentration: True/False, don't know what it is  

IsConcentration: True/False, don't know what it is  

HasModules: True/False, don't know what it is  

FullPartTimeDescription: don't know what it is  

FollowupStudiesDescription: it's in the name, describes that you can pursue in the field of the Bachelor without further requirement, has to be checked if you want to change field  

CombinationDescription: Says that in addition to major 150 you have to take minor 30 from the faculty or others (or the same faculty if it has to be the same)  

SRicht: Branch of Studies (business administration, informatics, business informatics)  

RequirementsDescription: don't know what it is

Organizations_OObjId  

Scs_ObjectId  

Head_ObjId  

Coordination_ObjId



In [ ]:
import requests
import pandas as pd

def fetch_cg_details_raw(
    CgObjId: str,
    PiqYear: str,
    PiqSession: str,
    language: str
) -> dict:
    """
    Fetches the raw JSON payload for CgDetailsSet(CgObjId, PiqYear, PiqSession)
    (with $expand=Cgs,Organizations,Head,Coordination,Scs) and returns the Python
    dictionary corresponding to the "d" node.

    Parameters
    ----------
    CgObjId : str
        Curriculum‐group ID (e.g. "51087264").
    PiqYear : str
        The PIQ year (e.g. "2024").
    PiqSession : str
        The PIQ session (e.g. "004").
    language : str
        Two‐letter language code, either "EN" or "DE".

    Returns
    -------
    dict
        The raw JSON dictionary under the "d" key. Must contain all scalar fields
        and all expanded navprops (Cgs, Organizations, Head, Coordination, Scs).

    Raises
    ------
    ValueError
        If `language` is not "EN" or "DE".
    requests.HTTPError
        If the HTTP request returns a non‐200 status code.
    KeyError
        If the JSON response is missing "d".
    """
    language = language.upper()
    if language not in ("EN", "DE"):
        raise ValueError("language must be 'EN' or 'DE'")

    # Construct the URL with $expand for all sub‐collections
    endpoint = (
        "https://studentservices.uzh.ch/sap/opu/odata/uzh/vvz_data_srv/"
        f"CgDetailsSet(CgObjId='{CgObjId}',PiqYear='{PiqYear}',PiqSession='{PiqSession}')"
        f"?sap-client=001"
        f"&$expand=Cgs,Organizations,Head,Coordination,Scs"
        f"&sap-language={language}"
        f"&$format=json"
    )

    response = requests.get(endpoint)
    response.raise_for_status()

    payload = response.json()
    if "d" not in payload:
        raise KeyError("Expected key 'd' not found in JSON response")

    return payload["d"]

def clean_cg_details(
    raw_d: dict,
    language: str,
    export_csv: bool = False
) -> pd.DataFrame:
    """
    Takes the raw 'd' dictionary returned by fetch_cg_details_raw(...) and
    “cleans” it into a single‐row DataFrame with columns:
      - CgObjId
      - PiqYear
      - PiqSession
      - MainLanguageDescription
      - FutherLanguageDescription
      - AdmissionReqDescription
      - Text
      - PointsText
      - CatText
      - CommonDescription
      - DegreeDescription
      - GradesDescription
      - VistasDescription
      - StructureDescription
      - RulesDescription
      - AdviceDescription
      - IsTextEditable
      - HasChildConcentration
      - IsConcentration
      - HasModules
      - FullPartTimeDescription
      - FollowupStudiesDescription
      - CombinationDescription
      - RequirementsDescription
      - SRicht
      - Organizations_OObjId   (comma‐separated IDs)
      - Scs_ObjectId           (comma‐separated IDs)
      - Head_ObjId             (comma‐separated IDs)
      - Coordination_ObjId     (comma‐separated IDs)

    If export_csv=True, writes a file named
      CgDetails_<language>_<CgObjId>.csv
    """
    language = language.upper()
    if language not in ("EN", "DE"):
        raise ValueError("language must be 'EN' or 'DE'")

    # Helper to safely pull a scalar string, defaulting to ""
    def _scalar(key: str) -> str:
        return raw_d.get(key, "") or ""

    # Helper to pull a boolean, defaulting to False
    def _bool(key: str) -> bool:
        return bool(raw_d.get(key, False))

    # 1) Mandatory scalar fields
    CgObjId_val   = _scalar("CgObjId")
    if not CgObjId_val:
        raise KeyError("Expected 'CgObjId' not found in raw data")
    PiqYear_val   = _scalar("PiqYear")
    PiqSession_val= _scalar("PiqSession")

    # 2) All the long‐text scalar fields
    main_lang_desc     = _scalar("MainLanguageDescription")
    futher_lang_desc   = _scalar("FutherLanguageDescription")
    admission_req_desc = _scalar("AdmissionReqDescription")
    text_field         = _scalar("Text")
    points_text        = _scalar("PointsText")
    cat_text           = _scalar("CatText")
    common_desc        = _scalar("CommonDescription")
    degree_desc        = _scalar("DegreeDescription")
    grades_desc        = _scalar("GradesDescription")
    vistas_desc        = _scalar("VistasDescription")
    structure_desc     = _scalar("StructureDescription")
    rules_desc         = _scalar("RulesDescription")
    advice_desc        = _scalar("AdviceDescription")

    # 3) The new scalar/boolean fields
    is_text_editable          = _bool("IsTextEditable")
    has_child_concentration   = _bool("HasChildConcentration")
    is_concentration          = _bool("IsConcentration")
    has_modules               = _bool("HasModules")
    full_part_time_desc       = _scalar("FullPartTimeDescription")
    followup_studies_desc     = _scalar("FollowupStudiesDescription")
    combination_desc          = _scalar("CombinationDescription")
    requirements_desc         = _scalar("RequirementsDescription")
    branch_of_studies         = _scalar("SRicht")

    # 4) Helper to join sub‐entity IDs into a comma‐separated string
    def _join_ids(nav_key: str, id_field: str) -> str:
        group = raw_d.get(nav_key, {})
        arr   = group.get("results", [])
        if not arr:
            return ""
        collected = [item.get(id_field, "") for item in arr if item.get(id_field, "")]
        return ",".join(collected) if collected else ""

    orgs_ids   = _join_ids("Organizations", "OObjId")
    scs_ids    = _join_ids("Scs",          "ObjectId")
    head_ids   = _join_ids("Head",         "Objid")
    coord_ids  = _join_ids("Coordination", "Objid")

    # 5) Build the one‐row dictionary
    row = {
        "CgObjId":                     CgObjId_val,
        "PiqYear":                     PiqYear_val,
        "PiqSession":                  PiqSession_val,
        "MainLanguageDescription":     main_lang_desc,
        "FutherLanguageDescription":   futher_lang_desc,
        "AdmissionReqDescription":     admission_req_desc,
        "Text":                        text_field,
        "PointsText":                  points_text,
        "CatText":                     cat_text,
        "CommonDescription":           common_desc,
        "DegreeDescription":           degree_desc,
        "GradesDescription":           grades_desc,
        "VistasDescription":           vistas_desc,
        "StructureDescription":        structure_desc,
        "RulesDescription":            rules_desc,
        "AdviceDescription":           advice_desc,
        "IsTextEditable":              is_text_editable,
        "HasChildConcentration":       has_child_concentration,
        "IsConcentration":             is_concentration,
        "HasModules":                  has_modules,
        "FullPartTimeDescription":     full_part_time_desc,
        "FollowupStudiesDescription":  followup_studies_desc,
        "CombinationDescription":      combination_desc,
        "RequirementsDescription":     requirements_desc,
        "Organizations_OObjId":        orgs_ids,
        "Scs_ObjectId":                scs_ids,
        "Head_ObjId":                  head_ids,
        "Coordination_ObjId":          coord_ids,
        "SRicht":                      branch_of_studies
    }

    df = pd.DataFrame([row])

    # 6) Export to CSV if requested
    if export_csv:
        filename = f"./csvs/CgDetails_{language}_{CgObjId_val}.csv"
        df.to_csv(filename, index=False)
        print(f"Cleaned DataFrame written to '{filename}'")

    return df

if __name__ == "__main__":
    """
    Example “main” routine:
      1. Prompts user (or simply hard‐codes) a CgObjId, PiqYear, PiqSession, language.
      2. Calls fetch_cg_details_raw(...) to retrieve the full JSON under "d".
      3. Calls clean_cg_details(...) to flatten into a DataFrame.
      4. Prints the DataFrame (and optionally saves CSV).
    """

    # 1) Set your parameters here (or replace with input() or argparse)
    CgObjId   = "51087264"
    PiqYear   = "2024"
    PiqSession= "004"
    language  = "EN"
    export_csv = True  # change to False if you do not want a CSV written

    # 2) Fetch the raw JSON dictionary
    try:
        raw_data = fetch_cg_details_raw(
            CgObjId    = CgObjId,
            PiqYear    = PiqYear,
            PiqSession = PiqSession,
            language   = language
        )
    except Exception as e:
        print("Error fetching raw data:", e)
        exit(1)

    # 3) Clean it into a DataFrame
    try:
        df_clean = clean_cg_details(
            raw_d      = raw_data,
            language   = language,
            export_csv = export_csv
        )
    except Exception as e:
        print("Error cleaning data:", e)
        exit(1)

    # 4) Print the resulting DataFrame
    print("\n=== Final DataFrame ===")
    print(df_clean.to_string(index=False))


Cleaned DataFrame written to './csvs/CgDetails_DE_51087264.csv'

=== Final DataFrame ===
 CgObjId PiqYear PiqSession MainLanguageDescription FutherLanguageDescription                                                                                                                                                                                                                                                                                  AdmissionReqDescription                     Text PointsText   CatText                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

#### FETCH ALL SM IDS(study module) WITH YEAR AND SESSION 
#### EXPORT AS CSV

In [ ]:
import requests
import pandas as pd

def fetch_sm_search_dataframe(
    top: int = 5000,
    csv_path: str = None
) -> pd.DataFrame:
    """
    Fetch all SMSearch entries (paginated) and return a DataFrame
    with columns [ObjectId, AcademicYear, AcademicPeriod].

    Parameters
    ----------
    top : int, optional
        How many records to request per page (default=5000).
    csv_path : str, optional
        If provided (e.g. "all_sm.csv"), the resulting DataFrame
        will be saved to this path as CSV (index=False).

    Returns
    -------
    pd.DataFrame
        One row per SMSearch entry, with only the three columns:
        ObjectId, AcademicYear, AcademicPeriod.
    """
    base_url = (
        "https://studentservices.uzh.ch/sap/opu/zodatav4/"
        "sap/zcm_vvz_v4_ui/srvd/sap/zsb_vvz/0001/SMSearch"
    )

    all_rows = []
    skip = 0

    while True:
        params = {
            "$skip": skip,
            "$top": top,
        }
        resp = requests.get(base_url, params=params)
        resp.raise_for_status()

        data = resp.json()
        page = data.get("value", [])
        if not page:
            break

        # extract fields
        for item in page:
            all_rows.append({
                "ObjectId":       item.get("ObjectId", ""),
                "AcademicYear":   item.get("AcademicYear", ""),
                "AcademicPeriod": item.get("AcademicPeriod", "")
            })

        skip += top

    df = pd.DataFrame(all_rows, columns=["ObjectId", "AcademicYear", "AcademicPeriod"])

    if csv_path:
        df.to_csv(csv_path, index=False)
        print(f"Exported {len(df)} rows to {csv_path}")

    return df


# Example usage:
if __name__ == "__main__":
    # fetch everything and save to "sm_search.csv"
    df = fetch_sm_search_dataframe(csv_path="./csvs/sm_search.csv")
    print("Done—total rows:", len(df))
    print(df.head())


#### FILTER DATAFRAME OF SMDETAILS BY OBJID AND/OR YEAR AND/OR SESSION

#### RETURNS FILTERED DATAFRAME

In [ ]:

def filter_search_df(
    df: pd.DataFrame,
    object_ids: list[str] = None,
    academic_year: str = None,
    session: str = None
) -> pd.DataFrame:
    """
    Filter a DataFrame with columns ['ObjectId','AcademicYear','AcademicPeriod'].

    Parameters
    ----------
    df : pd.DataFrame
        Original DataFrame to filter.
    object_ids : list[str], optional
        If provided, only rows whose ObjectId is in this list are kept.
    academic_year : str, optional
        If provided, only rows with AcademicYear == this value are kept.
    session : str, optional
        If provided, only rows with AcademicPeriod == this value are kept.

    Returns
    -------
    pd.DataFrame
        A copy of `df` filtered according to the given criteria.
        If no filter is provided, returns a copy of `df` unchanged.
    """
    mask = pd.Series(True, index=df.index)

    if object_ids is not None:
        mask &= df["ObjectId"].isin(object_ids)

    if academic_year is not None:
        mask &= df["AcademicYear"] == academic_year

    if session is not None:
        mask &= df["AcademicPeriod"] == session

    return df.loc[mask].copy()


# Example usage:
if __name__ == "__main__":

    # 4) Combine filters
    combined = filter_search_df(
        df,
        object_ids=["50018319"],
        academic_year="2024",
        session="003"
    )
    print("Combined filter:\n", combined)


#### FETCH ALL SM DETAILS FROM DATAFRAME (normally from the filtered version above)

#### EXPORT AS CSV

In [ ]:
def fetch_and_flatten_sm_details(
    search_df: pd.DataFrame,
    language: str = "EN",
    csv_path: str = None
) -> pd.DataFrame:
    """
    For each row in search_df (with ObjectId/AcademicYear/AcademicPeriod),
    fetch SmDetailsSet expanded on Partof, Organizations, Responsible,
    Events, Events/Persons, OfferPeriods; flatten all fields into columns;
    return a wide DataFrame. Optionally write to csv_path.
    """
    language = language.upper()
    if language not in ("EN","DE"):
        raise ValueError("language must be 'EN' or 'DE'")

    def _flatten(obj, prefix="", out=None):
        if out is None:
            out = {}
        if isinstance(obj, dict):
            if "__metadata" in obj:
                obj = {k:v for k,v in obj.items() if k!="__metadata"}
            if "results" in obj and isinstance(obj["results"], list):
                for i, sub in enumerate(obj["results"], 1):
                    _flatten(sub, f"{prefix}{i}_", out)
            else:
                for k,v in obj.items():
                    _flatten(v, f"{prefix}{k}_", out)
        elif isinstance(obj, list):
            for i, sub in enumerate(obj, 1):
                _flatten(sub, f"{prefix}{i}_", out)
        else:
            out[prefix[:-1]] = obj
        return out

    records = []
    for _, row in search_df.iterrows():
        obj, yr, ses = row["ObjectId"], row["AcademicYear"], row["AcademicPeriod"]
        url = (
            "https://studentservices.uzh.ch/sap/opu/odata/uzh/vvz_data_srv/"
            f"SmDetailsSet(SmObjId='{obj}',PiqYear='{yr}',PiqSession='{ses}')"
            "?sap-client=001"
            f"&$expand=Partof,Organizations,Responsible,Events,Events/Persons,OfferPeriods"
            f"&sap-language={language}"
            "&$format=json"
        )
        resp = requests.get(url); resp.raise_for_status()
        d = resp.json().get("d", {})
        records.append(_flatten(d, ""))

    flat_df = pd.DataFrame(records)

    if csv_path:
        flat_df.to_csv(csv_path, index=False)
        print(f"Flattened SM details written to {csv_path}")

    return flat_df


if __name__ == "__main__":
    # 1) Fetch the list of SM identifiers
    search_df = fetch_sm_search_dataframe()

    # 2) Fetch & flatten all details in English, export to CSV
    flat_df = fetch_and_flatten_sm_details(
        search_df = combined,
        language  = "EN",
        csv_path  = "./csvs/sm_details_flat.csv"
    )

    # 3) Inspect
    print("First few rows of the flattened SM details:")
    print(flat_df.head())


#### FETCH AN EVENT

#### RETURNS AS A CSV FILE

In [ ]:
import requests
import pandas as pd

def fetch_and_flatten_edetails(
    EObjId: str,
    PiqYear: str,
    PiqSession: str,
    language: str = "EN",
    csv_path: str = None
) -> pd.DataFrame:
    """
    Fetches EDetailsSet(EObjId,PiqYear,PiqSession) expanded on
    Rooms, Persons, Schedule, Schedule/Rooms, Schedule/Persons,
    Modules, Links; flattens everything into one wide row.

    Parameters
    ----------
    EObjId : str
        Event‐ID (e.g. "51250847")
    PiqYear : str
        PIQ year (e.g. "2024")
    PiqSession : str
        PIQ session (e.g. "004")
    language : str
        "EN" or "DE"
    csv_path : str, optional
        If provided, write the flattened DataFrame to this CSV.

    Returns
    -------
    pd.DataFrame
        One‐row DataFrame with every scalar and every sub‐element
        enumerated (e.g. Rooms1_RoomText, Persons1_FirstName, Schedule1_Date, etc.).
    """
    language = language.upper()
    if language not in ("EN", "DE"):
        raise ValueError("language must be 'EN' or 'DE'")

    # 1) Fetch raw JSON "d"
    url = (
        "https://studentservices.uzh.ch/sap/opu/odata/uzh/vvz_data_srv/"
        f"EDetailsSet(EObjId='{EObjId}',PiqYear='{PiqYear}',PiqSession='{PiqSession}')"
        "?sap-client=001"
        f"&$expand=Rooms,Persons,Schedule,Schedule/Rooms,Schedule/Persons,Modules,Links"
        f"&sap-language={language}"
        "&$format=json"
    )
    resp = requests.get(url)
    resp.raise_for_status()
    payload = resp.json().get("d", {})

    # 2) Recursive flattener
    def _flatten(obj, prefix="", out=None):
        if out is None:
            out = {}
        if isinstance(obj, dict):
            # drop metadata
            if "__metadata" in obj:
                obj = {k:v for k,v in obj.items() if k != "__metadata"}
            # OData nav with results
            if "results" in obj and isinstance(obj["results"], list):
                for i, sub in enumerate(obj["results"], 1):
                    _flatten(sub, f"{prefix}{i}_", out)
            else:
                for k,v in obj.items():
                    _flatten(v, f"{prefix}{k}_", out)
        elif isinstance(obj, list):
            for i, sub in enumerate(obj, 1):
                _flatten(sub, f"{prefix}{i}_", out)
        else:
            out[prefix[:-1]] = obj
        return out

    flat = _flatten(payload, "")

    df = pd.DataFrame([flat])

    # 3) Optional CSV export
    if csv_path:
        df.to_csv(csv_path, index=False)
        print(f"Flattened EDetails written to {csv_path}")

    return df


if __name__ == "__main__":
    # Example usage:
    EObjId    = "51250847"
    PiqYear   = "2024"
    PiqSession= "004"
    language  = "EN"
    csv_path  = f"./csvs/edetails_{language}_{EObjId}_flat.csv"

    df = fetch_and_flatten_edetails(
        EObjId     = EObjId,
        PiqYear    = PiqYear,
        PiqSession = PiqSession,
        language   = language,
        csv_path   = csv_path
    )
    print(df.head())


#### FETCH A PERSON

#### RETURNS AS A CSV FILE

In [ ]:
import os
import requests
import pandas as pd

def fetch_and_flatten_pdetails(
    Objid: str,
    PiqYear: str,
    PiqSession: str,
    language: str = "EN",
    export_csv: bool = False,
    csv_folder: str = None
) -> pd.DataFrame:
    """
    Fetches PDetailsSet(Objid,PiqYear,PiqSession) expanded on
    Responsible and Events, flattens all fields (including nested lists)
    into a single wide row, and returns it as a DataFrame.

    Parameters
    ----------
    Objid : str
        The personnel‐detail ID (e.g. "01051625").
    PiqYear : str
        The PIQ year (e.g. "2021").
    PiqSession : str
        The PIQ session (e.g. "003").
    language : str
        Two‐letter code "EN" or "DE".
    export_csv : bool
        If True, writes the flattened result to CSV.
    csv_folder : str, optional
        Folder in which to save the CSV (defaults to cwd).

    Returns
    -------
    pd.DataFrame
        A one‐row DataFrame containing every field under the "d" node,
        with nested arrays/enumerations flattened into columns.
    """
    language = language.upper()
    if language not in ("EN", "DE"):
        raise ValueError("language must be 'EN' or 'DE'")

    # 1) Fetch raw JSON
    url = (
        "https://studentservices.uzh.ch/sap/opu/odata/uzh/vvz_data_srv/"
        f"PDetailsSet(Objid='{Objid}',PiqYear='{PiqYear}',PiqSession='{PiqSession}')"
        "?sap-client=001"
        "&$expand=Responsible,Events"
        f"&sap-language={language}"
        "&$format=json"
    )
    resp = requests.get(url)
    resp.raise_for_status()
    raw = resp.json().get("d", {})

    # 2) Recursive flattener
    def _flatten(obj, prefix="", out=None):
        if out is None:
            out = {}
        if isinstance(obj, dict):
            # drop metadata if present
            if "__metadata" in obj:
                obj = {k: v for k, v in obj.items() if k != "__metadata"}
            # OData navprop with results
            if "results" in obj and isinstance(obj["results"], list):
                for i, sub in enumerate(obj["results"], 1):
                    _flatten(sub, f"{prefix}{i}_", out)
            else:
                for k, v in obj.items():
                    _flatten(v, f"{prefix}{k}_", out)
        elif isinstance(obj, list):
            for i, sub in enumerate(obj, 1):
                _flatten(sub, f"{prefix}{i}_", out)
        else:
            out[prefix[:-1]] = obj
        return out

    flat = _flatten(raw, "")
    df = pd.DataFrame([flat])

    # 3) Optional export with language and Objid in filename
    if export_csv:
        filename = f"PDetails_{language}_{Objid}.csv"
        if csv_folder:
            os.makedirs(csv_folder, exist_ok=True)
            path = os.path.join(csv_folder, filename)
        else:
            path = filename
        df.to_csv(path, index=False)
        print(f"Flattened PDetails written to {path}")

    return df


if __name__ == "__main__":
    # Example usage
    Objid      = "01051625"
    PiqYear    = "2021"
    PiqSession = "003"
    language   = "EN"

    df_p = fetch_and_flatten_pdetails(
        Objid        = Objid,
        PiqYear      = PiqYear,
        PiqSession   = PiqSession,
        language     = language,
        export_csv   = True,
        csv_folder   = "./csvs"
    )

    print("PDetails flattened row:")
    print(df_p.head(1).T)  # transpose for readability
